In [14]:
import pandas as pd
import numpy as np
import csv

from plotly import graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
import string

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


change input

In [62]:
# file as a user input
# should be chaged to integrtion to back end later
#change the input method to upload by user

import os
df = pd.read_csv('/content/drive/MyDrive/ML models/Daily_consumption_EL_FJV/Tallbacka/Tallbacka_EL_2023.csv')
df

,Period,Förbrukning
0,2022-12-31 23:00:00,12.044
1,2023-01-01 00:00:00,14.158
2,2023-01-01 01:00:00,13.412
3,2023-01-01 02:00:00,13.484
4,2023-01-01 03:00:00,13.458
...,...,...
7291,2023-10-31 18:00:00,15.588
7292,2023-10-31 19:00:00,13.104
7293,2023-10-31 20:00:00,12.600
7294,2023-10-31 21:00:00,9.540


In [63]:
# import os


# output_file = "/content/drive/MyDrive/ML models/Daily_consumption_EL_FJV/Tallbacka/Tallbacka_cleaned.csv"


# Rename columns from Swedish to English
column_mapping = {
    "Period": "Timestamp",  # Rename 'Period' to 'Timestamp'
    "Förbrukning": "Consumption"  # Rename 'Förbrukning' to 'Consumption'
}
df.rename(columns=column_mapping, inplace=True)

# Check for null values
print("Checking for null values...")
print(df.isnull().sum())

# Drop rows with null values
df.dropna(inplace=True)



#other df imputation techniques


# Ensure the Timestamp column is in datetime format
# Use `format` to explicitly parse the date and time
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Drop rows where Timestamp conversion failed (optional, for invalid rows)
df = df.dropna(subset=['Timestamp'])

# Filter rows for the year 2023
df = df[df['Timestamp'].dt.year == 2023]




# Convert the Timestamp column to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Extract the date from the Timestamp
df['Date'] = df['Timestamp'].dt.date

# Group by the Date and calculate the total consumption for each day-daily total consumption
df = df.groupby('Date')['Consumption'].sum().reset_index()
df

# # Save the cleaned data to a new CSV file
# df.to_csv(output_file, index=False)

# print(f"Cleaned data saved to: {output_file}")


Checking for null values...
Timestamp      0
Consumption    0
dtype: int64


,Date,Consumption
0,2023-01-01,481.476
1,2023-01-02,788.772
2,2023-01-03,879.993
3,2023-01-04,794.436
4,2023-01-05,809.388
...,...,...
299,2023-10-27,983.424
300,2023-10-28,429.504
301,2023-10-29,427.020
302,2023-10-30,730.260


Recommended method: Using API to retrive and merge historical weather df

Alternative Method

* 64.62374793689602, 21.20208994001161 Tallbacka förskola

* 64.75742763100038, 20.976930910623626 Norrbacka Förskola
* 64.9533930253045, 21.208040506944595 Byskeskolan
* 64.62114716001102, 21.20414726274176 Bureskolan
* 64.75336571800571, 21.012759213596993 Moröhöjdens förskola
* 64.51644049976487, 20.65190851614634 Björnåkersskolan

Change to filter school cordinate related weather data

In [68]:
# Define the input file path - change this line
input_file = '/content/drive/MyDrive/ML models/Daily_consumption_EL_FJV/Tallbacka/Tallbacka_EL_2023.csv'

# Extract the base name of the file (e.g., "Tallbacka_EL_2023.csv")
base_name = os.path.basename(input_file)  # Extracts "Tallbacka_EL_2023.csv"

# Extract "Tallbacka" from the base name (assuming it's the first part of the file name before '_')
stripped_name = base_name.split('_')[0]  # Keeps only "Tallbacka"

# Build the new file path based on the stripped name -- change this
directory = '/content/drive/MyDrive/ML models/weatherdata'
weather_file = f"{directory}/{stripped_name}.csv"

# Load the dataset
try:
    df1 = pd.read_csv(weather_file)
    print(f"Successfully loaded the Historical data: {weather_file}")
    print(df1.head())
except FileNotFoundError:
    print(f"Historical data not found: {weather_file}. Please check the path or input.")

Successfully loaded the Historical data: /content/drive/MyDrive/ML models/weatherdata/Tallbacka.csv
     time  weather_code (wmo code)  temperature_2m_max (°C)  \
0  1/1/23                       71                      2.9   
1  1/2/23                        3                     -2.2   
2  1/3/23                       71                     -9.0   
3  1/4/23                       71                     -8.1   
4  1/5/23                       71                     -5.5   

   daylight_duration (s)  rain_sum (mm)  snowfall_sum (cm)  \
0               14492.92            0.0               0.07   
1               14698.83            0.0               0.00   
2               14920.45            0.0               0.07   
3               15156.70            0.0               0.07   
4               15406.50            0.0               1.89   

   wind_speed_10m_max (km/h)  
0                       27.1  
1                       19.5  
2                       13.8  
3                       

In [73]:

# Convert the Date column in df to datetime format and reformat it
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True).dt.strftime('%d-%m-%Y')  # Change format to DD-MM-YYYY

# Load the weather dataframe from the specified file
# Replace `File` with the actual DataFrame loaded from the file path
weather_df = pd.read_csv(weather_file)  # Make sure `File` contains the correct file path

# Convert the time column in weather_df to datetime format and reformat it
weather_df['time'] = pd.to_datetime(weather_df['time'], format='%m/%d/%y')  # Adjust format if needed
weather_df['Date'] = weather_df['time'].dt.strftime('%d-%m-%Y')  # Change format to DD-MM-YYYY

# Drop the 'time' column in weather_df as it is no longer needed
weather_df = weather_df.drop(columns=['time'])

# Select only the necessary columns from the weather dataframe
weather_df = weather_df[['Date', 'weather_code (wmo code)', 'temperature_2m_max (°C)',
                         'daylight_duration (s)', 'rain_sum (mm)', 'snowfall_sum (cm)', 'wind_speed_10m_max (km/h)']]

# Merge the dataframes on the Date column, avoiding duplicate columns
merged_df = pd.merge(df, weather_df, on='Date', how='left')

# Save the merged dataframe to a new CSV
output_file = "/content/drive/MyDrive/ML models/Daily_consumption_EL_FJV/Tallbacka/Tallbacka_with_weather.csv"
merged_df.to_csv(output_file, index=False)

# Show the first few rows of the merged dataset
print(f"Merged dataset saved to {output_file}")
print(merged_df.head())



Merged dataset saved to /content/drive/MyDrive/ML models/Daily_consumption_EL_FJV/Tallbacka/Tallbacka_with_weather.csv
         Date  Consumption  weather_code (wmo code)  temperature_2m_max (°C)  \
0  01-01-2023      481.476                       71                      2.9   
1  02-01-2023      788.772                        3                     -2.2   
2  03-01-2023      879.993                       71                     -9.0   
3  04-01-2023      794.436                       71                     -8.1   
4  05-01-2023      809.388                       71                     -5.5   

   daylight_duration (s)  rain_sum (mm)  snowfall_sum (cm)  \
0               14492.92            0.0               0.07   
1               14698.83            0.0               0.00   
2               14920.45            0.0               0.07   
3               15156.70            0.0               0.07   
4               15406.50            0.0               1.89   

   wind_speed_10m_max (km/h

Fetch user input

In [ ]:
#downloadable csv to be downloaded by the user and to front end

In [ ]:
# if requested load visualization from backend

